# NBA Sleepers

## By: Carlos Villagomez & Kevin Tran

## CPSC 310, Spring 2019

## Introduction:
We are using a dataset consisting of two decades worth of NBA player data ranging from the 1996-2016 seasons on each player's seasonal statistics who was a member of a team's roster. We implemented several classification models including K-Nearest Neighbor (KNN), Sklearn KNN, Naive Bayes, Sklearn Naive Bayes, Linear Regression, and Sklearn Linear Regression to classify whether a player is a "sleeper" or not. We define a "sleeper" pick as a player whose net rating, which is a stat that evaluates a team's point differential per 100 possessions while he is on the court is greater than or equal to the average player of the next highest draft stock. For example, someone like Zach Randolph, who is predicted to be selected in the Top 5, but was actually selected in the late 1st round would be classified as a "sleeper". We found that the most accurate classification approach was KNN when predicting the "sleeper" picks.

## Data Analysis:
The dataset consists of 21 attribute values including each player's demographic variables (e.g. height, weight, age, etc.), biographical details (team, draft year, round, etc.), and basic box score statistics (avg points, rebounds, assists, etc.). There are 9,561 total instances of players, but only 1,892 unique players in the dataset. Now, there were initially 51 rows of missing data, but the author of the dataset handled this by manually filling the gaps using player data from the online database, Basketball Reference.

We have used several models for performing classification. This includes K-Nearest Neighbor (KNN), Sklearn KNN, Naive Bayes, Sklearn Naive Bayes Linear Regression, and Sklearn Linear Regression. It was found that KNN had the highest accuracy value when predicting a player's draft stock at , while the linear regression model predicted a player's stock at a mere level of accuracy of .

## Classification Results:


## Conclusion:
The dataset we used presented a few problems with analysis including  that the 51 rows that were initially missing data, but were handled by the author filled in the missing data from Basketball Reference. Also, because the statistics were only retrieved from the years 1996-2016, our results sometimes did not encompass the entirety of their careers. For instance, a player such as Joe Dumars, who is a hall-of-famer but retired in 1999 and thus our dataset only has data from 1996-1999 that does not encompass his best seasons in the NBA with the Detroit Pistons is only predicted as a lottery pick when he should really be labeled as a top 5 pick if you look at the entirety of his career when he was at his best during the late 1980's and early 1990's. The same goes for players who were drafted towards the end of our range and have only played a few years in the NBA before reaching the prime of their careers. An example of this would be Nikola Jokic who is now just playing the best basketball of his career after have just received his first All-Star selection in 2019 after being drafted in 2014 NBA draft as the 41st pick. However, his stats have significantly improved since his rooke season in 2015-2016 so while he is only predicted to be a lottery pick with our algorithm given the dataset we have, if we were to apply our algorithm containing statistics all the way up to the 2019 season he would likely be predicted to be a Top 5 rather than a lottery pick now that he is considered to be one of the best centers in the NBA.

We devloped a classification approach to determine whether a player is a "sleeper" based on whether their net rating is greater than or equal to  the average player of the next highest draft stock as described in the Introduction. Our linear regression classification model had a predictive accuracy of 0.802. Next, the Sklearn linear classification model had a predictive accuracy of 0.772. Our KNN classification model had a predictive accuracy of 0.675 with k = 13 clusters. Then our Naive Bayes model had a predictive accuracy of 0.689. 